# Facet spaces and hybrid methods (Unit 2.7)

Mixed methods for second order problems lead to saddle point problems, and indefinite matrices. By hybridization one obtains a positive definite system again. It's structure is similar to the non-conforming $\mathcal{P}^1$ method, but hybridization works for any order. See text-book by Brezzi and Fortin.

One skips the normal-continuity of the $H(div)$ variable, and reinforces it by a Lagrange parameter. This leads to the following discrete system:

Find $\sigma, u, \hat{u} \in \Sigma_h \times V_h \times F_h$:

$$\int \lambda^{-1} \sigma \tau + \sum_T \int_T \text{ div } \tau u - \sum_F \int_F [\tau \cdot n] \hat{u} = 0 \ \ \forall \tau \in \Sigma_h$$

$$\int \text{div } \sigma v = -\int fv \ \ \forall v \in V_h$$

$$\text{I added the sum here: }\sum_F \int_F [\sigma \cdot n] \hat{v} = \int_{\Gamma_n} g \hat{v} \ \ \forall \hat{v} \in F_h $$ 

where $\Sigma_h$ is a discontinuous $H(div)$ finite element space, $V_h$ a sub-space of $L_2$, and $F_h$ consists of polynomials on every edge.

### Notes: 
The notation above: $[\sigma n]\hat{v}$ indicates a 'jump' term.  We will avoid integrating jump terms on facets by integrating the boundaries of H(div) elements.  See below.

I think $F_h$ stands for Facet

We are solving the same diffusion problem as in Mixed. The main difference is that now we have $\hat{u}$ (edge polynomials) and $\sigma$ can be discontinous.  Open the Mixed notebook and compare!

The first equation here corresponds to 
$$ \int_\Omega \lambda^{-1}\sigma \cdot \tau + \int_\Omega \text{div}\tau\ u - \int_{\Gamma_D} u_D \tau_n = 0$$
which results from integration by parts on the first equation in Mixed.

The second equation here corresponds to the second equation in Mixed
$$ \int_\Omega \text{div} \sigma v = -\int_\Omega f v $$

The third equation here corresponds to $\sigma_n = g$ on $\Gamma_N$

**Note: I changed the signs of two terms above.  That made the equations above consistent with both the Mixed example and the code below.**

We can use the 'discontinuous' keyword for a finite element space to specify whether a space is discontinuous.  Read on...

In [1]:
from netgen.geom2d import unit_square
from ngsolve import *
import netgen.gui
%gui tk
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1)) # DD: changed from 0.2 for better comparison with Mixed


same example as in 'mixed':

In [2]:
source = sin(3.14*x)
ud = CoefficientFunction(5)
g = CoefficientFunction([y*(1-y) if bc=="left" else 0 for bc in mesh.GetBoundaries()])
lam = 10

define spaces:

- The discontinuous flag generates an element-wise $H(Div)$-space
- FacetFESpace lives only on facets (i.e. faces in 3D, edges in 2D, points in 1D)

Boundary conditions are now posed for the facet-space

In [3]:
order = 3
# DD: V corresponds to Sigma above, Q to V above for consistency with Mixed (a convention?)
V = HDiv(mesh, order=order, discontinuous = True)
Q = L2(mesh, order=order-1)
F = FacetFESpace(mesh, order=order, dirichlet="bottom")  # Note: We didn't have this dirichlet condition in Mixed
X = FESpace([V,Q,F])
print ("sigmadofs:", X.Range(0))
print ("udofs:    ", X.Range(1))
print ("uhatdofs: ", X.Range(2))


sigmadofs: slice(0, 4600, 1)
udofs:     slice(4600, 5980, 1)
uhatdofs:  slice(5980, 7440, 1)


Assemble forms. The jump-term is rewritten as

$$\sum_F \int_F [\sigma_n] v = \sum_T \int_{\partial T} \sigma_n v$$


Ok, so sum of integrals of jumps on facets equals sum of integrals over element boundaries... 

This was confusing until I added the sum in the third equation at the top

Is the jump term $[\sigma_n]=\sigma_{n_{left}}−\sigma_{n_{right}}$?



In [4]:
sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

condense=True

a = BilinearForm(X, eliminate_internal=condense)
a += SymbolicBFI(1/lam * sigma*tau + div(sigma)*v + div(tau)*u)
n = specialcf.normal(mesh.dim)
a += SymbolicBFI(sigma*n*vhat-tau*n*uhat, element_boundary=True) # DD: changed sign of sigma*n*vhat to +

# Compare with mixed -- we've added an integrator on the element boundaries
# am = BilinearForm(fesm)
# am += SymbolicBFI(1/lam * sigma * tau + div(sigma)*v + div(tau)* u)
# am.Assemble()

c = Preconditioner(a, "bddc")

f = LinearForm(X)
f += SymbolicLFI(-source*v)
f += SymbolicLFI(g*vhat.Trace(), BND) # DD: changed sign to +
# if we omit Trace() we will get error assembling:
# RuntimeError: cannot evaluate facet-fe inside element.  This makes sense because specifying BND
# means integrate over boundary elements -- including their interiors.

# Compare with mixed! -- we don't need the ud *tau integrator for the Dirichlet boundary now.
# We added an integrator on the Facet space.

# fm = LinearForm(fesm)
# fm += SymbolicLFI(-source*v)
# normal = specialcf.normal(mesh.dim)
# fm += SymbolicLFI(ud * (tau.Trace()*normal), BND)
# fm.Assemble()
a.Assemble()
print ("A non-zero", a.mat.nze)

gfu = GridFunction(X)


A non-zero 27920


Solve system. Leave everything to the sparse direct solver, or use CG

In [5]:
f.Assemble()
gfu.components[2].Set(ud, BND)

if condense:
    f.vec.data += a.harmonic_extension_trans * f.vec
    
    solvers.CG(mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, initialize=False)
    
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * f.vec
else:
    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gfu.vec
    inv = a.mat.Inverse(freedofs=X.FreeDofs())
    gfu.vec.data += inv * r
    


RuntimeError: cannot evaluate facet-fe inside element, add trans simdin Assemble LinearForm


In [ ]:
Draw (gfu.components[0], mesh, "sigma")
Draw (gfu.components[1], mesh, "u")